In [1]:
import os
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers


In [2]:
train_df = pd.read_csv("./data/train.csv")
test_df = pd.read_csv("./data/test.csv")
print("Train shape : ",train_df.shape)
print("Test shape : ",test_df.shape)

Train shape :  (1306122, 3)
Test shape :  (375806, 2)


In [3]:
print(pd.DataFrame.info(test_df))
print(pd.DataFrame.info(train_df))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375806 entries, 0 to 375805
Data columns (total 2 columns):
qid              375806 non-null object
question_text    375806 non-null object
dtypes: object(2)
memory usage: 5.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1306122 entries, 0 to 1306121
Data columns (total 3 columns):
qid              1306122 non-null object
question_text    1306122 non-null object
target           1306122 non-null int64
dtypes: int64(1), object(2)
memory usage: 29.9+ MB
None


In [4]:
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=0)

In [5]:
embed_size = 300 # how big is each word vector
max_features = 50000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a question to use

## fill up the missing values
train_X = train_df["question_text"].values
val_X = val_df["question_text"].values
test_X = test_df["question_text"].values

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

## Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

## Get the target values
train_y = train_df['target'].values
val_y = val_df['target'].values

In [6]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size)(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 1

In [7]:
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))

Train on 1175509 samples, validate on 130613 samples
Epoch 1/2
1175509/1175509 [==============================] - 257s 219us/sample - loss: 0.1218 - acc: 0.9534 - val_loss: 0.1058 - val_acc: 0.9581
Epoch 2/2
1175509/1175509 [==============================] - 251s 213us/sample - loss: 0.0970 - acc: 0.9613 - val_loss: 0.1071 - val_acc: 0.9576


In [8]:
pred_noemb_val_y = model.predict([val_X], batch_size=1024, verbose=1)
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_noemb_val_y>thresh).astype(int))))



130613/130613 [==============================] - 7s 53us/sample
F1 score at threshold 0.1 is 0.5579091494845361
F1 score at threshold 0.11 is 0.5673577269348311
F1 score at threshold 0.12 is 0.5756694075479656
F1 score at threshold 0.13 is 0.5837283461173834
F1 score at threshold 0.14 is 0.5910269367667091
F1 score at threshold 0.15 is 0.5968666069829901
F1 score at threshold 0.16 is 0.6015112891478515
F1 score at threshold 0.17 is 0.6071197411003237
F1 score at threshold 0.18 is 0.6119094580633042
F1 score at threshold 0.19 is 0.6158533682456558
F1 score at threshold 0.2 is 0.6203038340969376
F1 score at threshold 0.21 is 0.6238523149052551
F1 score at threshold 0.22 is 0.6286053529906496
F1 score at threshold 0.23 is 0.6318422896027218
F1 score at threshold 0.24 is 0.6346475024039678
F1 score at threshold 0.25 is 0.6372388365424007
F1 score at threshold 0.26 is 0.6388860130448286
F1 score at threshold 0.27 is 0.6416692814559147
F1 score at threshold 0.28 is 0.6449053010263465
F1 scor

In [9]:
pred_noemb_test_y = model.predict([test_X], batch_size=1024, verbose=1)

375806/375806 [==============================] - 19s 51us/sample
